In [71]:
wikiglove_dict = {}

file = open('/Users/kylesargent/Desktop/GLoVe + VerbNet/glove.6B.300d.txt')
for row in file:
    key = row[:row.index(' ')]
    value = list(map(lambda x : float(x), str.split(row[row.index(' ') + 1:])))
    wikiglove_dict[key] = value
    
print("Stored WikiGLoVe dictionary.")    

KeyboardInterrupt: 

In [5]:
twitterglove_dict = {}

file = open('/Users/kylesargent/Desktop/GLoVe + VerbNet/glove.twitter.27B.100d.txt')
for row in file:
    key = row[:row.index(' ')]
    value = list(map(lambda x : float(x), str.split(row[row.index(' ') + 1:])))
    twitterglove_dict[key] = value
    
print("Stored TwitterGLoVe dictionary.")

In [10]:
import csv
import json
reader = csv.reader(open('/Users/kylesargent/Desktop/GLoVe + VerbNet/verb_frames.csv', 'r'))
verbnet_dict = {}
for row in reader:
    k, v = row
    verbnet_dict[k] = json.loads(v)
    
print("Stored VerbNet dictionary.")

Stored VerbNet dictionary.


In [11]:
import numpy as np

def si(dct, key): #safe dictionary indexing procedure
    try: return(dct[key]) 
    except KeyError: return(False)
    
gen_dict = {}
for verb, verbnet_vec in verbnet_dict.items():
    v = si(wikiglove_dict, verb)
    w = si(twitterglove_dict, verb)
    if (not v == False and not w == False):
        gen_dict[verb] = (verbnet_vec, np.hstack((v,w)))
    
#There are 3988 verbs in the intersection of glove and verbnet
print("Stored intersecting verbs in VerbNet and GLoVe.")

Stored intersecting verbs in VerbNet and GLoVe.


In [29]:
import numpy as np
import random
    
dat = list(gen_dict.items())
l = round(.8 * len(dat))
random.shuffle(dat)
X = [value[1] for _key, value in dat]
y = [value[0] for _key, value in dat]

X_train = np.array(X[:l])
y_train = np.array(y[:l])

X_validation = np.array(X[l:])
y_validation = np.array(y[l:])

In [69]:
#Creating MatLab-compatible data

import scipy.io

train_data = (X_train - X_train.min(0)) / X_train.ptp(0)
train_target = (2 * y_train - 1).T

test_data = (X_validation - X_validation.min(0)) / X_validation.ptp(0)
test_target = (2 * y_validation - 1).T

scipy.io.savemat('train_data.mat', mdict={'train_data': train_data})
scipy.io.savemat('train_target.mat', mdict={'train_target': train_target})
scipy.io.savemat('test_data.mat', mdict={'test_data': test_data})
scipy.io.savemat('test_target.mat', mdict={'test_target': test_target})


In [70]:
BPMLL_predict_proba = scipy.io.loadmat("Outputs.mat")["Outputs"].T

loss1 = count_unpredicted(sloppy_predict(X_validation, BPMLL_predict_proba, 60), y_validation)/len(y_validation)
print(loss1)

0.9715061058344641


In [32]:
from sklearn.neural_network import MLPClassifier
from sknn.mlp import Classifier, Layer, Convolution
from IPython.display import clear_output

clf_mlp = MLPClassifier(activation = 'relu', 
                        batch_size = 200, 
                        solver='adam', 
                        alpha=1e-5,
                        hidden_layer_sizes=(8000,), 
                        random_state=2, 
                        early_stopping=True, 
                        validation_fraction = .15)

nn = Classifier(
    layers=[
        Convolution("Rectifier", channels=1, kernel_shape=(20,20)),
        Layer("Rectifier", units=4000),
        Layer("Sigmoid")],
    learning_rate=0.002,
    weight_decay=1e-5,
    valid_size = .1,
    batch_size = 200,
    debug=True,
    n_iter=100)
#nn.fit(X_train, y_train)
#clf_mlp.fit(X_train, y_train)

loss1 = count_unpredicted(sloppy_predict(X_validation, clf_mlp, 60), y_validation)/len(y_validation)
clear_output()
print(loss1)

  - Reshaping input array from (2949, 400) to (2654, 1, 20, 20).


KeyboardInterrupt: 

In [67]:
def sloppy_predict(X, predictions, n):
    '''
    predictions:
        predictions = BPMLL_predict_proba
            for matlab model
        predictions = model.predict_proba(X)
            for sklearn model
        predictions = np.array([1 - X.T[0] for X in nn.predict_proba(X_validation)]).T 
            for sknn models
    '''
    for vec in predictions:
        ceil = np.sort(vec)[::-1][n]
        for i in range(0, len(vec)):
            if vec[i] > ceil:
                vec[i] = 1
            else:
                vec[i] = 0
    return(predictions)

def count_unpredicted(X_predicted, X_actual):
    arr = X_actual - X_predicted
    total = 0
    for i in arr.flatten():
        if i == 1:
            total += 1
    return(total)

In [177]:
#Method 1. We apply PCA twice to get a direct sum decomposition of the GLoVe vector space into meaning and syntax
import numpy as np
import random
from sklearn.decomposition import PCA
import matplotlib.pyplot

SymSyn_pairs = []
for key1, v1 in gen_dict.items():
    for key2, v2 in gen_dict.items():
        if (key1 > key2 and v1[0] == v2[0]):
            SymSyn_pairs += [(np.array(v1[1]), np.array(v2[1]))]

print("Stored similar syntax pairs.")
            
DiffSyn_pairs = []
for key1, v1 in gen_dict.items():
    for key2, v2 in gen_dict.items():
        if (key1 > key2 and v1[0] != v2[0] and random.uniform(0,1) > .99):
            DiffSyn_pairs += [(np.array(v1[1]), np.array(v2[1]))]

print("Stored different syntax pairs.")

Meaning_vecs = []
for v1, v2 in SymSyn_pairs:
    Meaning_vecs += [list(v1 - v2)]
    
Regularization_vecs = []
for v1, v2 in DiffSyn_pairs:
    Regularization_vecs += [list(v1 - v2)]

pca_meaning = PCA(n_components = meaningspace_size)
pca_meaning.fit(Meaning_vecs)

#Gradient descent on basis extension s_1 ... s_l to the syntax space

import random
import copy
from scipy.optimize import approx_fprime
from functools import reduce

S = np.array(copy.copy(Meaning_vecs)).T
#np.shape(S) = (DIM, len(Syn_pairs))
SST = np.dot(S, S.T)

G = np.array(copy.copy(Regularization_vecs)).T
#np.shape(S) = (DIM, len(Syn_pairs))
GGT = np.dot(G, G.T)

P = np.identity(DIM)
for i in range(0,meaningspace_size):
    P[i][i] = 0.0
    
P_syntax = P
P_meaning = np.identity(DIM) - P_syntax
    
def rand_m():
    Mt = list(pca_meaning.components_)
    for i in range(0, syntaxspace_size):
        Mt += list(np.array(2*np.random.rand(1,DIM) - 1))
    Mt = np.array(Mt)
    M = Mt.T
    return(M)
    
def loss(M):
    n = np.sum((reduce(np.dot, [M, P, np.linalg.inv(M), S]))**2)
    d = np.sum((reduce(np.dot, [M, P, np.linalg.inv(M), G]))**2)
    return(np.log(n/d))

def grad(M, i,j):
    u = np.sum((reduce(np.dot, [M, P, np.linalg.inv(M), S]))**2)
    v = np.sum((reduce(np.dot, [M, P, np.linalg.inv(M), G]))**2)
    
    M_inv = np.linalg.inv(M)
    P1u = 2 * reduce(np.dot, [M, P, M_inv, SST])
    P1v = 2 * reduce(np.dot, [M, P, M_inv, GGT])
    
    Eij = np.zeros((DIM, DIM))
    Eij[i][j] = 1.0
            
    P2 = reduce(np.dot, [Eij, P, M_inv]) + reduce(np.dot, [M,P,-M_inv, Eij, M_inv])
    uprime = np.trace(np.dot(P1u.T, P2))
    vprime = np.trace(np.dot(P1v.T, P2))
    return( (uprime / u) - (vprime / v) )

def itrdescent_stochastic(M, eta):
    for i in range(0, DIM):
        for j in range(meaningspace_size, DIM):
            M[i][j] -= eta * grad(M, i,j)
            
def itrdescent_nonstochastic(M, eta):
    u = np.sum((reduce(np.dot, [M, P, np.linalg.inv(M), S]))**2)
    v = np.sum((reduce(np.dot, [M, P, np.linalg.inv(M), G]))**2)
    
    M_used = copy.copy(M)
    M_inv = np.linalg.inv(M)
    P1u = 2 * reduce(np.dot, [M, P, M_inv, SST])
    P1v = 2 * reduce(np.dot, [M, P, M_inv, GGT])
    Eij = np.zeros((DIM, DIM))
    for i in range(0, DIM):
        for j in range(meaningspace_size, DIM):
            Eij[i][j] = 1.0
            P2 = reduce(np.dot, [Eij, P, M_inv]) + reduce(np.dot, [M_used,P,-M_inv, Eij, M_inv])
            uprime = np.trace(np.dot(P1u.T, P2))
            vprime = np.trace(np.dot(P1v.T, P2))
            M[i][j] -= eta * ((uprime / u) - (vprime / v))
            Eij[i][j] = 0.0
    

def anneal():
    best_M = rand_m()
    best_loss = loss(best_M)
    
    for i in range(0, 10000):
        M = rand_m()
        M_loss = loss(M)
        if best_loss > M_loss:
            best_loss = M_loss
            best_M = M
    return(best_loss, M)

def fixed_descent(M, eta, iters):
    for i in range(0, iters):
        if (i % 50 == 0):
            print(loss(M))
        descent(M, eta)

def variable_descent(M, eta, iters, stochastic=False):
    incident = 0
    loss_new = loss(M)
    for i in range(0,iters):
        loss_old = loss_new
        print('Explanatory proportion:', np.exp(loss_old) * len(DiffSyn_pairs) / len(SymSyn_pairs))
        M_old = copy.copy(M)
        if stochastic:
            itrdescent_stochastic(M, eta)
        else:
            itrdescent_nonstochastic(M, eta)
        loss_new = loss(M)
        if loss_new > loss_old:
            eta /= 2
            M = M_old
            incident = 0
            print('Eta decreased to', eta)
        elif loss_new > .99 * loss_old and incident > 20:
            eta *= 2
            print('Eta increased to', eta)
        incident += 1
        
def anneal():
    best = rand_m()
    best_loss = loss(best)
    
    for i in range(0, 100):
        M = rand_m()
        M_loss = loss(M)
        if best_loss > M_loss:
            best_loss = M_loss
            best = M
    return(best_loss, best)

#Defining subspace projections and reconstructing GLoVe vectors from syntax and meaning spaces

def meaning(v):
    M_inv = np.linalg.inv(M)
    return(reduce(np.dot, [M, P_meaning, M_inv, v]))
    
def syntax(v):
    M_inv = np.linalg.inv(M)
    return(reduce(np.dot, [M, P_syntax, M_inv, v]))

In [182]:
M = res[1]
variable_descent(M, 1, 0, stochastic=False)

In [48]:
#Testing a hypothesis: Any two GLoVe vectors with corresponding identical VerbNet frame vectors are more cosine similar in general
import numpy as np

def cossim(v,w):
    return(np.dot(v,w)/(np.sqrt(sum(v**2) * sum(w**2))))

total_all = 0
total_simframe = 0
num_all = 0
num_simframe = 0
t = 0
l = len(gen_dict.items())

for key1, v1 in gen_dict.items():
    for key2, v2 in gen_dict.items():
        if (key1 > key2):
            v = cossim(v1[1], v2[1])
            total_all += v
            num_all += 1
            if (v1[0] == v2[0]):
                total_simframe += v
                num_simframe += 1
            
print(total_simframe/num_simframe)
print(total_all/num_all)

#Results for 300d GLoVe
#.10624
#.04691
#Compelling evidence that VerbNet and GLoVe vectors have a special relationship!
#Analysis: On average, syntactically similar vectors have more than twice as (cosine) similar GLoVe vectors
#A good sanity check that more investigation is warranted...

0.106239521638
0.0469074223333


In [28]:
for i in range(0,7):
    print([1,2,3,4,5,6,7][-i] == [1,2,3,4,5,6,7][::-1][i-1])

True
True
True
True
True
True
True
